In [2]:
import torch 
from ipsl_dataset import surface_variables,IPSL_DCPP,plev_variables
import numpy as np

In [3]:
train = IPSL_DCPP('test',generate_statistics=True)
train_dataloader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True)



def z_normalize(data:np.ndarray,axes:tuple):
    mean = np.nanmean(data,axis=axes,keepdims=False)
    std_dev = np.nanstd(data,axis=axes,keepdims=False)
    # mean = np.broadcast_to(mean,data.shape)
    # std_dev = np.broadcast_to(std_dev,data.shape)
    # standardized_data = (data - mean) / std_dev
    return mean,std_dev

In [4]:
surface_means = []
plev_means = []

surface_stds = []
plev_stds = []
# datas = []
for count in range(1000):
    # x,y,t1,t2 = batch
    batch = next(iter(train_dataloader))
    surface = batch['state_surface'].squeeze()
    plev = batch['state_level'].squeeze()
    
    # datas.append(x)
    print(count)
    i_mean,i_std = z_normalize(surface.numpy(),(1,2))
    t_mean,t_std = z_normalize(plev.numpy(),(1,2,3))
    surface_means.append(i_mean)
    surface_stds.append(i_std)
    plev_means.append(t_mean)
    plev_stds.append(t_std)
surface_means_out = np.nanmean(np.array(surface_means),axis=0)
surface_stds_out = np.nanmean(np.array(surface_stds),axis=0)
plev_means_out = np.nanmean(np.array(plev_means),axis=0)
plev_stds_out = np.nanmean(np.array(plev_stds),axis=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
np.save('data/surface_means_test',surface_means_out)
np.save('data/surface_stds_test',surface_stds_out)
np.save('data/plev_means_test',plev_means_out)
np.save('data/plev_stds_test',plev_stds_out)

In [4]:
stacked = np.stack(datas)

In [5]:
stacked.shape

(10, 136, 143, 144)

In [6]:
np.array(means).shape

(10, 136)

In [7]:
np.load('std_devs.npy')

array([2.74373341e+01, 3.53500099e+01, 1.34439974e+01, 9.45808664e-02,
       5.02852321e-01, 4.06214863e-01, 1.13858409e-01, 4.71004695e-02,
       7.52755404e-01, 4.18635346e-02, 1.13169622e+00, 1.69927371e+00,
       3.58024955e+00, 1.51866484e+01, 8.21689415e+00, 7.27038878e-06,
       6.37716630e-06, 1.86779681e-09, 4.40111370e-09, 1.05015019e-08,
       2.81162702e-08, 2.13032646e+01, 1.52532840e+00, 1.27155045e+02,
       3.34994656e-05, 2.01305520e-05, 1.67795288e+02, 7.84410226e-09,
       1.27610118e-08, 2.49755883e-09, 2.34500463e-09, 6.31195540e-09,
       6.37740095e-06, 5.82076609e-09, 1.12430367e-08, 0.00000000e+00,
       9.02046260e-09, 8.78637456e-06, 2.78358631e+01, 1.61617165e+01,
       2.15115795e+01, 3.34921926e-01, 1.75342914e-02, 2.54790955e+01,
       8.71168673e-02, 2.47866083e-05, 6.18532219e+01, 2.56314068e+01,
       1.45131426e+01, 5.78049384e-03, 3.50952505e-05, 2.59288972e-05,
       8.72585588e-06, 1.48088808e+01, 9.52940137e+03, 1.06032434e+03,
      

In [6]:
np.nanmean(np.array(std_devs),axis=0)

array([0.52326447, 0.52326846, 0.5230063 , 0.52335125, 0.52302456,
       0.5229055 , 0.52360886, 0.5251202 , 0.52319324, 0.5239608 ,
       0.5232958 , 0.52326167, 0.5233174 , 0.52302617, 0.52371144,
       0.5406014 , 0.5186203 , 0.50004727, 0.5251411 , 0.52808094,
       0.5351337 , 0.5232961 , 0.5255908 , 0.5273697 , 0.5244022 ,
       0.53295463, 0.522011  , 0.54562515, 0.5389302 , 0.5420583 ,
       0.5389902 , 0.52543753, 0.51862913, 0.5379356 , 0.5279912 ,
       0.53130925, 0.        , 0.52594286, 0.5420642 , 0.5233633 ,
       0.52337706, 0.523335  , 0.9988474 , 0.9997422 , 1.0060253 ,
       1.0011673 , 1.0010872 , 1.0010636 , 0.9982065 , 1.0025305 ,
       0.99903685, 0.9986113 , 0.99641967, 1.0019749 , 0.9987043 ,
       0.99968874, 0.98943317, 1.0005232 , 0.9998635 , 1.0001446 ,
       1.0004922 , 1.000277  , 1.0189067 , 1.0189686 , 1.0172659 ,
       1.0396899 , 1.0384327 , 1.0331126 , 1.0372727 , 1.0090816 ,
       0.9992389 , 1.0001417 , 1.0010449 , 0.9993535 , 0.99305

In [38]:
example = np.array(std_devs)[:,0]

In [39]:
ns = np.zeros(10) 

In [40]:
ns[:] = 20000

In [41]:
np.power((((ns - 1) * np.power(example, 2)).sum() / (ns-1).sum()),1/2)

27.434968050351873

In [42]:
def pooled_standard_deviation(std_devs, sample_sizes):
    # Check if the lengths of std_devs and sample_sizes are the same
    if len(std_devs) != len(sample_sizes):
        raise ValueError("The lengths of std_devs and sample_sizes must be the same")

    # Calculate the pooled standard deviation
    numerator = np.sum((sample_sizes - 1) * np.square(std_devs))
    denominator = np.sum(sample_sizes - 1)
    pooled_std_dev = np.sqrt(numerator / denominator)

    return pooled_std_dev


In [43]:
pooled_standard_deviation(example,ns)

27.43496860653218

In [37]:
# np.nanmean(stacked,axis=(0,2,3),keepdims=True).flatten()
np.nanstd(stacked,axis=(0,2,3),keepdims=True).flatten()

array([2.74349670e+01, 3.53473053e+01, 1.34683065e+01, 9.47383642e-02,
       5.02782524e-01, 4.06380713e-01, 1.13592565e-01, 4.65145446e-02,
       7.52374947e-01, 4.19500917e-02, 1.13189268e+00, 1.69919646e+00,
       3.57755399e+00, 1.52129622e+01, 8.29440784e+00, 7.95205506e-06,
       6.40511462e-06, 1.48377366e-09, 4.27330615e-09, 1.02439026e-08,
       3.02476941e-08, 2.12974091e+01, 1.54097509e+00, 1.28200745e+02,
       3.35349650e-05, 2.22251801e-05, 1.67519547e+02, 9.09886211e-09,
       1.43189824e-08, 2.83535861e-09, 2.58306687e-09, 6.54988241e-09,
       6.40512144e-06, 6.45606368e-09, 1.13663017e-08, 1.69795733e-08,
       0.00000000e+00, 8.78796591e-09, 9.70986366e-06, 2.78150291e+01,
       1.61476822e+01, 2.14998188e+01, 3.32811952e-01, 1.77256074e-02,
       2.57896557e+01, 8.73388276e-02, 2.49866243e-05, 6.23638878e+01,
       2.68216534e+01, 1.48464708e+01, 5.70764393e-03, 3.52693351e-05,
       2.63229613e-05, 8.38321102e-06, 1.45853186e+01, 9.49853418e+03,
      

In [3]:
import xarray as xr
import os
store_dir = os.environ['STORE'] 
year = 1960
variation = 1
Amon =  xr.open_mfdataset(f'{store_dir}/s{year}-r{variation}i1p1f1/Amon/*.nc',compat='minimal')


In [2]:
from pytorch_lightning.loggers import WandbLogger
